In [1]:
import os
from openai import OpenAI
client = OpenAI()

client.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
assistant = client.beta.assistants.create(
    name="수학 선생님",
    instructions="당신은 친절한 수학선생님입니다. 사용자가 질문하는 수학문제에 대답하기 위해 code를 작성하고 실행하세요.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o-mini"
)
assistant

Assistant(id='asst_66T8rRXwoBDAdyZZDrhf8Bg3', created_at=1737535513, description=None, instructions='당신은 친절한 수학선생님입니다. 사용자가 질문하는 수학문제에 대답하기 위해 code를 작성하고 실행하세요.', metadata={}, model='gpt-4o-mini', name='수학 선생님', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)

In [3]:
thread = client.beta.threads.create()
thread

Thread(id='thread_wPAm8c9iEqSbtpcy1zIjGUTH', created_at=1737535518, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [4]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="`3x + 11 = 14`라는 방정식을 풀려고 하는데 도와줄 수 있어?"
)

In [5]:
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.data)

[Message(id='msg_t6dSyrFg08HPpE9ZozIYYAcI', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='`3x + 11 = 14`라는 방정식을 풀려고 하는데 도와줄 수 있어?'), type='text')], created_at=1737535526, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_wPAm8c9iEqSbtpcy1zIjGUTH')]


In [6]:
from typing_extensions import override
from openai import AssistantEventHandler
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

In [7]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="수학문제에 대해 설명하고 답을 같이 말해줘.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > 물론입니다! 방정식 \(3x + 11 = 14\)을 풀어보겠습니다.

1. **양쪽에서 11을 빼기:** 먼저 11을 제거하여 \(x\)를 구할 준비를 합니다.
   \[
   3x + 11 - 11 = 14 - 11
   \]
   그러면,
   \[
   3x = 3
   \]

2. **양쪽을 3으로 나누기:** 이제 양쪽을 3으로 나눕니다.
   \[
   x = \frac{3}{3}
   \]
   그러면,
   \[
   x = 1
   \]

결과적으로, 방정식 \(3x + 11 = 14\)의 해는 \(x = 1\)입니다.